I want to try and explain my way through a simple backpropogation implementation. Let $\sigma$ be the sigmoid function

$\displaystyle\sigma(x) = \frac{1}{1 + e^{-x}}$

We will consider a very simple single layer neural network that is predicting a membership in a single class. Our input data are in $\mathbb{R}^n$ and so we have the following set up. 

$x \in \mathbb{R}^n$, $y \in \mathbb{R}$, $W \in M_{n, 1}(\mathbb{R})$, $b \in \mathbb{R}$. Where $x$ is a single input data, $y$ is a single target label (in $\{0, 1\}$), $W$ is the matrix of parameters for our predictor and b is our bias term. 

The form of our prediction function is then:

$\displaystyle F(x) = \sigma(W\cdot x + b)$

Now fix a batch of input data $X = (x^1, x^2, \dots, x^k)$ with labels $(y^1, y^2, \dots, y^k)$. If we use cross entropy loss then our loss function over this batch of data is:

$\displaystyle J(W, b) = - \sum_i y^i\log(F(x^i))$

(Maybe we actually take the average of this? It does not change the calculation as multiplying by a constant $k$ will just rescale every term in our calculations.)

# Derivatives

Remember that our goal here is to compute $\displaystyle \frac{\partial J}{\partial w_i}$ for each $i = 1, \dots, n$ and $\displaystyle \frac{\partial J}{\partial b}$.
If we let 

$\displaystyle f_W(x) = W \cdot x + b$

Then we can factor J nicely so that the chain rule tell us how to compute these derivatives.

I will switch to prime notation for notational convenience. Since we just add up the loss terms for each element in $X$, we can go ahead and assume that $b = 1$ and write $x = x^1, y=y^1$. Then $J(W, b) = - y \log(F_W(x))$.

Now $\displaystyle J' = (-y \log (F_W(x)))' = -y \cdot \frac{1}{F_W(x)} \cdot (F_W(x))'$ where we have used the chain rule in a straightforward way.

We have factored $F_W(x) = \sigma(f_W(x))$ so we can further unwrap the term

$(F_W(x))' = \sigma'(f_W(x)) \cdot f_W'(x)$.

Now depending on which variable we are using to take the derivative here (switching to prime notation has obscured this momentarily) we can easily compute $f_W'$.

We have $f_W' = x_i$ if we are taking the derivative wrt $w_i$ and $f_W' = 1$ if we are taking the derivative wrt $b$. 

(For this maybe it is helpful to remember that $x \in \mathbb{R}^n$ and so $x = (x_1, \dots, x_n)$.

Putting this all together and wrangling the mixing of dimensions in all of these calculations, we get:

$\nabla J = -y \cdot \frac{1}{F_W(x)} \cdot \sigma'(f_W(x)) \cdot (x_1, \dots, x_n, 1)^T$

We can clean this up a bit by writing $\hat{y} = F_W(x)$ and recalling that $\sigma'(z) = \sigma(z) ( 1 - \sigma(z))$. Also for convenience, let $\tilde{x} = (x_1, \dots, x_n, 1)$. Then we have 

$\nabla J = \frac{-y}{\hat{y}} \cdot \hat{y} (1 - \hat{y}) \cdot \tilde{x}$

If we do not have batch size 1 then what happens?  Well in this case some of these operations are just vectorized and we sum down the batch axis at the end.

$\nabla J = \displaystyle \sum_i\frac{-y^i}{\hat{y^i}} \cdot \hat{y^i} (1 - \hat{y^i}) \cdot \tilde{x^i}$ (or this divided by batch size for average loss).

# General Back prop
I am not going to flush out exactly how general backprop works (the details still get kind of hairy for me :O) but it is worth pointing out how the terms in this finally gradient computation get rolled up into a more general multi layer computation.

What we do is compute the gradient for the parameters in each layer one at a time and decompose the computation into several pieces, some of which have already been computed in our initial pass forward (needed to compute the loss initially!). The key is that the gradient for parameters in earlier layers depends on the gradient for parameters in later layers but these later gradients are independent of earlier layers. This means that we can get away with murder by computing the gradient for parameters in the deepest layers first and then incrementally step back to earlier layers. This is harder to see when there is only one layer :D.

The computation of the gradient for the parameters in each layer is decomposed into three pieces:

1. back propogated error terms from deeper in the network
2. activation term at this layer
3. linear term at this layer

In our setup above we have:

1. deeper errors (last layer) - $\displaystyle \frac {y}{\hat{y}}$
2. activation term at this layer - $\hat{y}(1 - \hat{y})$
3. linear term at this layer - $\tilde{x}$

The book-keeping gets a little hairier but eh. . .not that bad probably.

If we were to add another layer then we would have to compute a similar product of three terms but now term 1 would be replaced by this quantity that we are computing itself!

Note that if we use a different activation function, the second term will change somewhat. Hopefully the activation function used is such that the derivative it is still very easy to express in terms of the value of the function itself (which we have already computed in forward prop). For relu for example this is the case as the derivative is just

$ (\text{relu})'(x) = 1 \text{ if } x > 0 \text{ else } 0$  ( i guess just blink and ignore the case when $x = 0$ :P

If we use a different loss function then the initial error term that we roll backwards will be different but the way that we incrementally accumulate new terms as we move back through the layers will not change. 